# Transformations

All the Transformations are available here https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations
and https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html#pyspark.RDD


### map
Return a new distributed dataset formed by passing each element of the source through a function func.

In [0]:
rdd = sc.parallelize(["b", "a", "c"])
sorted(rdd.map(lambda x: (x, 1)).collect())


Out[1]: [('a', 1), ('b', 1), ('c', 1)]

### mapPartitions

Similar to map, but runs separately on each partition (block) of the RDD, so func must be of type Iterator<T> => Iterator<U> when running on an RDD of type T.

In [0]:
rdd = sc.parallelize([1, 8, 2, 3, 4, 5, 6], 2)

def f(iterator): 
    yield sum(iterator)

rdd.mapPartitions(f).collect()


Out[2]: [11, 18]

In [0]:
rdd = sc.parallelize([1, 8, 2, 3, 4, 5, 6], 2)

def f(iterator): 
    for i in iterator:
        yield i*2

rdd.mapPartitions(f).collect()

Out[3]: [2, 16, 4, 6, 8, 10, 12]

### mapPartitionsWithIndex

Similar to mapPartitions, but also provides func with an integer value representing the index of the partition, so func must be of type (Int, Iterator<T>) => Iterator<U> when running on an RDD of type T.

In [0]:
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): 
    data = 0
    for i in iterator:
        data += i
    yield (f'data = {data} and index = {splitIndex}')


rdd.mapPartitionsWithIndex(f).collect()


Out[4]: ['data = 1 and index = 0',
 'data = 2 and index = 1',
 'data = 3 and index = 2',
 'data = 4 and index = 3']

In [0]:
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(splitIndex, iterator): 
    data = 0
    for i in iterator:
        data += i
    yield (f'data = {data} and index = {splitIndex}')


rdd.mapPartitionsWithIndex(f).collect()

Out[5]: ['data = 3 and index = 0', 'data = 7 and index = 1']

### flatMap

Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [0]:
rdd = sc.parallelize([2, 3, 4])

rdd.flatMap(lambda x: range(1, x)).collect()


Out[6]: [1, 1, 2, 1, 2, 3]

In [0]:
rdd.flatMap(lambda x: [(x, x), (x, x)]).collect()


Out[7]: [(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]

### flatMapValues

Pass each value in the key-value pair RDD through a flatMap function without changing the keys; this also retains the original RDD’s partitioning.

In [0]:
rdd = sc.parallelize([("a", ["x", ["y", "z"]]), ("b", ["p", "r"])])

def f(x): 
    return x
    
rdd.flatMapValues(f).collect()


Out[8]: [('a', 'x'), ('a', ['y', 'z']), ('b', 'p'), ('b', 'r')]

In [0]:
rdd.flatMapValues(f).flatMapValues(f).collect()

Out[9]: [('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

### mapValues

Pass each value in the key-value pair RDD through a map function without changing the keys; this also retains the original RDD’s partitioning.


In [0]:
rdd = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): 
    return len(x)

rdd.mapValues(f).collect()


Out[10]: [('a', 3), ('b', 1)]

### filter

Return a new dataset formed by selecting those elements of the source on which func returns true.

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()


Out[11]: [2, 4]

### sample

Sample a fraction fraction of the data, with or without replacement, using a given random number generator seed.

In [0]:
rdd = sc.parallelize(range(100), 4)
rdd.sample(False, 0.1, 81).collect()


Out[12]: [4, 26, 39, 41, 42, 52, 63, 76, 80, 86, 97]

In [0]:
fractions = {"a": 0.2, "b": 0.1}
rdd = sc.parallelize(fractions.keys()).cartesian(sc.parallelize(range(0, 5)))
rdd.collect()

Out[13]: [('a', 0),
 ('a', 1),
 ('a', 2),
 ('a', 3),
 ('a', 4),
 ('b', 0),
 ('b', 1),
 ('b', 2),
 ('b', 3),
 ('b', 4)]

In [0]:
rdd.sampleByKey(False, fractions, 2).collect()

Out[14]: [('a', 1), ('a', 2), ('b', 4)]

### union

Return a new dataset that contains the union of the elements in the source dataset and the argument.

In [0]:
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()


Out[15]: [1, 1, 2, 3, 1, 1, 2, 3]

### intersection 

Return a new RDD that contains the intersection of elements in the source dataset and the argument.


In [0]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()


Out[16]: [1, 2, 3]

### distinct

Return a new dataset that contains the distinct elements of the source dataset.

In [0]:
sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())


Out[17]: [1, 2, 3]

### repartition
Reshuffle the data in the RDD randomly to create either more or fewer partitions and balance it across them. This always shuffles all data over the network.

In [0]:
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)
sorted(rdd.glom().collect())



Out[18]: [[1], [2, 3], [4, 5], [6, 7]]

In [0]:

rdd.repartition(2).glom().collect()


Out[19]: [[4, 5, 6, 7], [1, 2, 3]]

In [0]:

rdd.repartition(5).glom().collect()


Out[20]: [[], [], [], [6, 7], [1, 2, 3, 4, 5]]

### repartitionAndSortWithinPartitions

Repartition the RDD according to the given partitioner and, within each resulting partition, sort records by their keys. This is more efficient than calling repartition and then sorting within each partition because it can push the sorting down into the shuffle machinery.

In [0]:
rdd = sc.parallelize([(0, 5), (3, 8), (2, 6), (0, 8), (3, 8), (1, 3)])
rdd2 = rdd.repartitionAndSortWithinPartitions(2, lambda x: x % 2, True)
rdd2.glom().collect()


Out[21]: [[(0, 5), (0, 8), (2, 6)], [(1, 3), (3, 8), (3, 8)]]

### groupByKey
When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs.

Note: If you are grouping in order to perform an aggregation (such as a sum or average) over each key, using reduceByKey or aggregateByKey will yield much better performance.

Note: By default, the level of parallelism in the output depends on the number of partitions of the parent RDD. You can pass an optional numPartitions argument to set a different number of tasks.


In [0]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 4)])
rdd.groupByKey().collect()


Out[22]: [('a', <pyspark.resultiterable.ResultIterable at 0x7f9a10e551f0>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x7f9a10dfd0d0>)]

In [0]:
list(rdd.groupByKey().collect()[0][-1])

Out[23]: [1, 4]

In [0]:
rdd.groupByKey().mapValues(len).collect()

Out[24]: [('a', 2), ('b', 1)]

In [0]:

rdd.groupByKey().mapValues(list).collect()


Out[25]: [('a', [1, 4]), ('b', [1])]

### reduceByKey

When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [0]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 4)])
sorted(rdd.reduceByKey(add).collect())


Out[26]: [('a', 5), ('b', 1)]

### aggregateByKey

When called on a dataset of (K, V) pairs, returns a dataset of (K, U) pairs where the values for each key are aggregated using the given combine functions and a neutral "zero" value. Allows an aggregated value type that is different than the input value type, while avoiding unnecessary allocations. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [0]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2), ("a", 2), ("a", 2), ("a", 2), ("b", 4)])
seqFunc = (lambda x, y: (x[0] + y, x[1] + 1))
combFunc = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

rdd.aggregateByKey((0, 0), seqFunc, combFunc).collect()


Out[27]: [('a', (9, 5)), ('b', (5, 2))]

### sortByKey

When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the boolean ascending argument.

In [0]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
sc.parallelize(tmp).sortByKey().first()


Out[28]: ('1', 3)

In [0]:

sc.parallelize(tmp).sortByKey(True, 1).collect()


Out[29]: [('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]

In [0]:

sc.parallelize(tmp).sortByKey(True, 2).collect()


Out[30]: [('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]

In [0]:

tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
sc.parallelize(tmp2).sortByKey(True, 3, keyfunc=lambda k: k.lower()).collect()


Out[31]: [('a', 3),
 ('fleece', 7),
 ('had', 2),
 ('lamb', 5),
 ('little', 4),
 ('Mary', 1),
 ('was', 8),
 ('white', 9),
 ('whose', 6)]

### join

When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through leftOuterJoin, rightOuterJoin, and fullOuterJoin.

In [0]:
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2), ("a", 3)])
sorted(rdd1.join(rdd2).collect())


Out[32]: [('a', (1, 2)), ('a', (1, 3))]

### cogroup

When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. This operation is also called groupWith.


In [0]:
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2), ("a", 5), ("b", 6), ("c", 7)])
rdd1.cogroup(rdd2).collect()

Out[33]: [('b',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e69f70>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e70b50>)),
 ('c',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e70910>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e70df0>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e70ac0>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e70be0>))]

In [0]:
list(rdd1.cogroup(rdd2).collect())

Out[34]: [('b',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e04730>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e04b50>)),
 ('c',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e04820>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e042b0>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x7f9a10e047c0>,
   <pyspark.resultiterable.ResultIterable at 0x7f9a10e04df0>))]

In [0]:
[(x, tuple(map(list, y))) for x, y in sorted(list(rdd1.cogroup(rdd2).collect()))]

Out[35]: [('a', ([1], [2, 5])), ('b', ([4], [6])), ('c', ([], [7]))]

### cartesian

When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements).

In [0]:
rdd = sc.parallelize([1, 2])
rdd.cartesian(rdd).collect()


Out[36]: [(1, 1), (1, 2), (2, 1), (2, 2)]

### pipe

Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the process's stdin and lines output to its stdout are returned as an RDD of strings.

In [0]:
sc.parallelize(['1', '2', '', '3']).pipe('echo 1').collect()


Out[37]: ['1', '1', '1', '1', '1', '1', '1', '1']

### coalesce

Decrease the number of partitions in the RDD to numPartitions. Useful for running operations more efficiently after filtering down a large dataset.

In [0]:
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()


Out[38]: [[1], [2, 3], [4, 5]]

In [0]:

sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(1).glom().collect()


Out[39]: [[1, 2, 3, 4, 5]]